In [4]:
import pysam
import os
import pandas as pd
from pyfaidx import Fasta
from collections import *
from tools.intervals import *
from tools.misc import *
from tools.procOps import *
from tools.fileOps import *
from tools.bio import *
from tools.psl import *
from itertools import *
import bisect
import argparse


In [104]:
class MsaRecord(object):
    """Convenience holder for alignment information"""
    def __init__(self, msa_start, msa_stop, qname, seq):
        self.msa_start = msa_start
        self.msa_stop = msa_stop
        self.qname = qname
        self.seq = seq


def find_closest(numeric_list, query_number):
    """
    Given a list of numbers, and a single query number, find the number in the sorted list that is numerically
    closest to the query number. Uses list bisection to do so, and so should be O(log n)
    """
    sorted_numeric_list = sorted(numeric_list)
    pos = bisect.bisect_left(sorted_numeric_list, query_number)
    if pos == 0:
        return sorted_numeric_list[0]
    if pos == len(sorted_numeric_list):
        return sorted_numeric_list[-1]
    before = sorted_numeric_list[pos - 1]
    after = sorted_numeric_list[pos]
    if after - query_number < query_number - before:
        return after
    else:
        return before


def find_overlap(b1, b2, spacer=50):
    delta = b1.msa_stop - b2.msa_start
    b1_seq = b1.seq[-(delta + spacer):]
    b2_seq = b2.seq[:delta + spacer]
    aln = perform_aln(b1_seq, b2_seq)
    if len(aln) == 0:
        s = b1.seq[:-delta]
        assert len(s) > 0
        return s
    else:
        psl = sorted([PslRow(x.split('\t')) for x in aln], key=lambda x: x.coverage)[-1]
        cutoff = psl.query_coordinate_to_target(psl.q_end - 1) + 1
        return b1.seq[:-(cutoff - spacer)]


def merge_same(b1, b2):
    with TemporaryFilePath() as f, TemporaryFilePath() as f2:
        with open(f, 'w') as outf:
            write_fasta(outf, 'b1', b1.seq)
            write_fasta(outf, 'b2', b2.seq)
        cmd = ['muscle', '-in', f, '-out', f2]
        run_proc(cmd)
        fa = Fasta(f2)
        seq = []
        for x, y in zip(*[fa['b1'], fa['b2']]):
            if x == '-':
                seq.append(y)
            else:
                seq.append(x)
        return ''.join(seq)


def perform_aln(b1_seq, b2_seq):
    with TemporaryFilePath() as b1_f, TemporaryFilePath() as b2_f:
        with open(b1_f, 'w') as b1_f_h:
            write_fasta(b1_f_h, 'b1', b1_seq)
        with open(b2_f, 'w') as b2_f_h:
            write_fasta(b2_f_h, 'b2', b2_seq)
        cmd = ['blat', b1_f, b2_f, '-noHead', '/dev/stdout']
        return call_proc_lines(cmd)[:-2]


def construct_hg38_map(n2nl_aln, hg38_bam):
    """Constructs a map of hg38 position -> sequence alignment position -> MSA position"""
    # construct sequence alignment position -> MSA position map using the MSA
    aln_f = Fasta(n2nl_aln)
    seq_aln_map = defaultdict(dict)
    for name, seq in aln_f.iteritems():
        seq_pos = 0
        for aln_pos, x in enumerate(str(seq)):
            seq_aln_map[name][seq_pos] = aln_pos
            if x != '-':
                seq_pos += 1

    # find maximum position for reversing negative strand
    max_pos = {x: max(y.keys()) for x, y in seq_aln_map.iteritems()}

    # construct a hg38 -> sequence positions using the sequences trivially mapped back to hg38
    hg38_map = {}
    for rec in pysam.Samfile(hg38_bam):
        m = {y: x for x, y in rec.aligned_pairs}
        # invert positions for negative strand genes
        if rec.qname in ['NOTCH2', 'NOTCH2NL-A', 'NOTCH2NL-B']:
            m = {x: max_pos[rec.qname] - y for x, y in m.iteritems()}
        hg38_map[rec.qname] = m

    # construct a table mapping each alignment position to all hg38 positions
    r = defaultdict(dict)
    for name, pos_map in hg38_map.iteritems():
        for hg38_pos, seq_pos in pos_map.iteritems():
            aln_pos = seq_aln_map[name][seq_pos]
            r[name][aln_pos] = hg38_pos

    # now invert this map, so that we have our hg38 -> aln map
    final_map = {}
    for name in r:
        for aln_pos in r[name]:
            hg38_pos = r[name][aln_pos]
            assert hg38_pos not in final_map
            final_map[hg38_pos] = aln_pos

    return final_map


def load_alignments(bam, name, regions_of_interest, position_map, sorted_positions):
    """
    1) Load all alignments of contigs to hg38
    2) filter for those that overlap the MSA
    3) sort them by MSA positions based on the position map
    4) Filter for entirely overlapping
    5) return sorted lists of MsaRecords
    """
    msa_blocks = []
    for aln in pysam.Samfile(bam):
        if aln.is_unmapped:
            continue
        # make sure this alignment overlaps notch
        start = aln.reference_start
        end = aln.reference_end
        c = ChromosomeInterval('chr1', start, end, '.')
        # filter for overlapping our regions and not being too short
        if not interval_not_intersect_intervals(regions_of_interest, c) and len(c) > 100:
            # find the closest positions in hg38 that are present in our MSA
            closest_start = find_closest(sorted_positions, start)
            closest_stop = find_closest(sorted_positions, end)
            msa_start = position_map[closest_start]
            msa_stop = position_map[closest_stop]
            if msa_start > msa_stop:  # handle negative strand
                msa_start, msa_stop = msa_stop, msa_start
                seq = reverse_complement(aln.seq)
            else:
                seq = aln.seq
            b = MsaRecord(msa_start, msa_stop, aln.qname, seq)
            msa_blocks.append(b)

    # sort by MSA position
    sorted_msa_blocks = sorted(msa_blocks, key=lambda x: x.msa_start)

    # filter blocks for those that are entirely a subset of another
    # this removes misassemblies
    intervals = [ChromosomeInterval('', x.msa_start, x.msa_stop, '.', x) for x in sorted_msa_blocks]
    bad_intervals = set()
    for i1, i2 in combinations(intervals, 2):
        if i1.proper_subset(i2):
            bad_intervals.add(i1)
        elif i2.proper_subset(i1):
            bad_intervals.add(i2)
    filtered_msa_blocks = [x.data for x in intervals if x not in bad_intervals]

    return filtered_msa_blocks


def scaffold_alignments(filtered_msa_blocks):
    """
    Takes a sorted list of MsaRecord objects and scaffolds them
    """
    seq = []
    # construct a pairwise iterator over the filtered_msa_blocks
    a, b = tee(filtered_msa_blocks)
    _ = next(b, None)
    # keep track of sequences that got merged, so we don't merge them again
    merged = set()
    for i, (b1, b2) in enumerate(izip(a, b)):
        if b1.qname in merged:
                continue
        elif b1.qname == b2.qname:
            seq.append(merge_same(b1, b2))
            merged.add(b1.qname)
        elif b2.msa_start < b1.msa_stop:  # we have an overlap, resolve via pairwise alignment
            seq.append(find_overlap(b1, b2))
        elif b1.msa_stop == b2.msa_start:  # exactly contiguous, just add b1 and continue
            seq.append(b1.seq)
        else:  # we have a unknown gap, so add b1 then a gap
            seq.append(b1.seq)
            seq.append(''.join(['N'] * 100))
        # add the final b2 sequence
    seq.append(b2.seq)
    return ''.join(seq)

In [24]:
n2nl_aln = '/hive/users/ifiddes/notch_brian_viz/try_bwa/notch2nl_alignment.fa'
hg38_bam = '/hive/users/ifiddes/notch_brian_viz/try_bwa/hg38_mapped.bam'
n2_regions = '/hive/users/ifiddes/notch_brian_viz/try_bwa/n2_regions.bed'
bam = 'hg38_mapped_contigs/C1.hg38.bam'

In [23]:
# load the intervals we are interested in
start_stop_positions = {x.split()[3]: (x.split()[1], x.split()[2]) for x in open(n2_regions)}
start_stop_positions = {x: map(int, y) for x, y in start_stop_positions.iteritems()}

# load the map of hg38 positions to alignment positions
position_map = construct_hg38_map(n2nl_aln, hg38_bam)
# construct a sorted list of hg38 positions
sorted_positions = sorted(position_map.keys())

# construct our regions of interest
regions_of_interest = []
for start, stop in start_stop_positions.itervalues():
    regions_of_interest.append(ChromosomeInterval('chr1', start, stop, '.'))


In [25]:
filtered_msa_blocks = load_alignments(bam, name, regions_of_interest, position_map, sorted_positions)

In [28]:
for x in filtered_msa_blocks:
    print x.msa_start, x.msa_stop, x.qname

26145 119800 contig-100_0
119902 120700 contig-100_71
120676 121608 contig-100_61
121509 129212 contig-100_5
129269 153395 contig-100_2
153399 156152 contig-100_20
156179 157814 contig-100_29
157792 162896 contig-100_8
162873 163827 contig-100_52
163795 166955 contig-100_15
166856 168246 contig-100_44
168146 196751 contig-100_1
196727 205462 contig-100_4
205438 208034 contig-100_14


In [113]:
seq = []
# construct a pairwise iterator over the filtered_msa_blocks
a, b = tee(filtered_msa_blocks)
_ = next(b, None)
# keep track of sequences that got merged, so we don't merge them again
merged = set()
with open('test.fa', 'w') as outf:
    for i, (b1, b2) in enumerate(izip(a, b)):
        if b1.qname in merged:
                continue
        elif b1.qname == b2.qname:
            seq.append(merge_same(b1, b2))
            merged.add(b1.qname)
            print 'merged', b1.qname, b2.qname
        elif b2.msa_start < b1.msa_stop:  # we have an overlap, resolve via pairwise alignment
            s = find_overlap(b1, b2)
            seq.append(s)
            print 'overlap', b1.qname, b2.qname
            write_fasta(outf, b1.qname + b2.qname, ''.join([s[-200:], b2.seq[:200]]))
        elif b1.msa_stop == b2.msa_start:  # exactly contiguous, just add b1 and continue
            seq.append(b1.seq)
            print 'contiguous', b1.qname, b2.qname
        else:  # we have a unknown gap, so add b1 then a gap
            seq.append(b1.seq)
            seq.append(''.join(['N'] * 100))
            print 'gap', b1.qname, b2.qname
            write_fasta(outf, b1.qname + b2.qname, ''.join([b1.seq[-200:], b2.seq[:200]]))
        # add the final b2 sequence
    seq.append(b2.seq)

gap contig-100_0 contig-100_71
overlap contig-100_71 contig-100_61
overlap contig-100_61 contig-100_5
gap contig-100_5 contig-100_2
gap contig-100_2 contig-100_20
gap contig-100_20 contig-100_29
overlap contig-100_29 contig-100_8
overlap contig-100_8 contig-100_52
overlap contig-100_52 contig-100_15
overlap contig-100_15 contig-100_44
overlap contig-100_44 contig-100_1
overlap contig-100_1 contig-100_4
overlap contig-100_4 contig-100_14


In [2]:
# first, construct a map of sequence positions to alignment positions
aln_f = Fasta('notch2nl_alignment.fa')
seq_aln_map = defaultdict(dict)
for name, seq in aln_f.iteritems():
    seq_pos = 0
    for aln_pos, x in enumerate(str(seq)):
        seq_aln_map[name][seq_pos] = aln_pos
        if x != '-':
            seq_pos += 1

In [3]:
# find maximum position for reversing negative strand
max_pos = {x: max(y.keys()) for x, y in seq_aln_map.iteritems()}

In [4]:
# next, construct a map of hg38 positions to sequence positions using the alignment
hg38_map = {}
for rec in pysam.Samfile('hg38_mapped.bam'):
    m = {y: x for x, y in rec.aligned_pairs}
    if rec.qname in ['NOTCH2', 'NOTCH2NL-A', 'NOTCH2NL-B']:
        m = {x: max_pos[rec.qname] - y for x, y in m.iteritems()}
    hg38_map[rec.qname] = m

In [5]:
# construct a table mapping each alignment position to all hg38 positions
r = defaultdict(dict)
for name, pos_map in hg38_map.iteritems():
    for hg38_pos, seq_pos in pos_map.iteritems():
        aln_pos = seq_aln_map[name][seq_pos]
        r[name][aln_pos] = hg38_pos

df = pd.DataFrame.from_dict(r)
df.head()

,NOTCH2,NOTCH2NL-A,NOTCH2NL-B,NOTCH2NL-C,NOTCH2NL-D
0,120189999.0,NaN,NaN,149328817.0,NaN
1,120189998.0,NaN,148801426.0,149328818.0,NaN
2,120189997.0,NaN,148801425.0,149328819.0,NaN
3,120189996.0,NaN,148801424.0,149328820.0,NaN
4,120189995.0,NaN,148801423.0,149328821.0,NaN


In [6]:
# now invert this map, so that we have our hg38 -> aln map
final_map = {}
for name in r:
    for aln_pos in r[name]:
        hg38_pos = r[name][aln_pos]
        assert hg38_pos not in final_map
        final_map[hg38_pos] = aln_pos

In [7]:
# load the intervals we are interested in
start_stop_positions = {x.split()[3]: (x.split()[1], x.split()[2]) for x in open('n2_regions.bed')}
start_stop_positions = {x: map(int, y) for x, y in start_stop_positions.iteritems()}

# construct our regions of interest
regions_of_interest = []
for start, stop in start_stop_positions.itervalues():
    regions_of_interest.append(ChromosomeInterval('chr1', start, stop, '.'))

In [8]:
# load all the alignments, by qname
aln_map = {}
base_dir = '/hive/users/ifiddes/notch2nl_berkeley_data/E2del19N_E2del68_combined_longranger/E2del68_E2del19N_combined/new-assembly/hg38_mapped_contigs'
for name in ['A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2', 'N1', 'N2']:
    s = os.path.join(base_dir, '{}.hg38.bam'.format(name))
    aln_map[name] = list(pysam.Samfile(s))

In [73]:
# extract all alignments, discarding those not in our ROIs
blocks = {}
for name in aln_map:
    b = []
    for aln in aln_map[name]:
        if aln.is_unmapped:
            continue
        start = aln.reference_start
        end = aln.reference_end
        c = ChromosomeInterval('chr1', start, end, '.')
        if not interval_not_intersect_intervals(regions_of_interest, c) and len(c) > 100:
            b.append([start, end, aln.qname, aln.seq, aln.qstart, aln.qstart + aln.alen])
    blocks[name] = b

In [74]:
for b in blocks['A2']:
    print b[0], b[1], b[2], b[4], b[5]

120126469 120136856 contig-100_4 7 10394
146148508 146159995 contig-100_7 3 11490
146159966 146192832 contig-100_1 54 32920
146207843 146252183 contig-100_0 0 44340
146252085 146257333 contig-100_11 0 5248
146286025 146288109 contig-100_20 0 2084
148643514 148658579 contig-100_6 48 15113
148708073 148736282 contig-100_2 0 28209
148745798 148755309 contig-100_4 0 9511
148755275 148779513 contig-100_3 32 24270
148769779 148769881 contig-100_160 257 359
149332986 149335419 contig-100_16 0 2433
149335395 149350493 contig-100_5 56 15154


In [109]:
# for each alignment, determine the interval in the MSA
# use a closest finding algorithm to deal with unaligned regions

class Record(object):
    def __init__(self, msa_start, msa_stop, qname, seq, seq_start, seq_end):
        self.msa_start = msa_start
        self.msa_stop = msa_stop
        self.qname = qname
        self.seq = seq
        self.seq_start = seq_start
        self.seq_end = seq_end

def find_closest(numeric_list, query_number):
    """
    Given a list of numbers, and a single query number, find the number in the sorted list that is numerically
    closest to the query number. Uses list bisection to do so, and so should be O(log n)
    """
    sorted_numeric_list = sorted(numeric_list)
    pos = bisect.bisect_left(sorted_numeric_list, query_number)
    if pos == 0:
        return sorted_numeric_list[0]
    if pos == len(sorted_numeric_list):
        return sorted_numeric_list[-1]
    before = sorted_numeric_list[pos - 1]
    after = sorted_numeric_list[pos]
    if after - query_number < query_number - before:
        return after
    else:
        return before

sorted_positions = sorted(final_map.keys())

msa_blocks = {}
for name, b in blocks.iteritems():
    mb = []
    for start, end, qname, seq, seq_start, seq_end in b:
        closest_start = find_closest(sorted_positions, start)
        closest_stop = find_closest(sorted_positions, end)
        msa_start = final_map[closest_start]
        msa_stop = final_map[closest_stop]
        if msa_start > msa_stop:  # handle negative strand
            msa_start, msa_stop = msa_stop, msa_start
            seq = reverse_complement(seq)
        # TODO: Record doesn't need seq_start or seq_end
        with TemporaryFilePath() as tmp_f:
            write_fasta(tmp_f, qname, seq)
            cmd = ['faPolyASizes', tmp_f, '/dev/stdout']
            r = call_proc_lines(cmd)
        r = r[0].split()
        start = int(r[-2])
        stop = int(r[-1])
        seq = seq[start:len(seq) - stop]
        assert len(seq) > 0
        mb.append(Record(msa_start, msa_stop, qname, seq, seq_start, seq_end))
    # sort these to be in notch2nl order
    mb = sorted(mb, key=lambda x: x.msa_start)
    msa_blocks[name] = mb

In [110]:
for x in msa_blocks['A2']:
    print x.msa_start, x.msa_stop, x.qname, x.seq_start, x.seq_end, len(x.seq)

4172 6605 contig-100_16 0 2433 2411
6581 25896 contig-100_5 56 15154 15207
26147 50433 contig-100_3 32 24270 24230
35809 35911 contig-100_160 257 359 359
50399 60617 contig-100_4 0 9511 9497
53285 67570 contig-100_4 7 10394 17154
67552 69636 contig-100_20 0 2084 2067
70162 98420 contig-100_2 0 28209 28206
98405 103668 contig-100_11 0 5248 5248
103570 148362 contig-100_0 0 44340 44344
148367 163827 contig-100_6 48 15113 15162
163793 196751 contig-100_1 54 32920 32883
196722 208040 contig-100_7 3 11490 11428


In [111]:
# filter blocks for those that are entirely a subset of another
# this removes misassemblies
filtered_blocks = {}
for name, mb in msa_blocks.iteritems():
    intervals = [ChromosomeInterval('', x.msa_start, x.msa_stop, '.', x) for x in mb]
    bad_intervals = set()
    for i1, i2 in combinations(intervals, 2):
        if i1.proper_subset(i2):
            bad_intervals.add(i1)
        elif i2.proper_subset(i1):
            bad_intervals.add(i2)
    filtered_blocks[name] = [x.data for x in intervals if x not in bad_intervals]

In [112]:
for x in filtered_blocks['A2']:
    print x.msa_start, x.msa_stop, x.qname, x.seq_start, x.seq_end, len(x.seq)

4172 6605 contig-100_16 0 2433 2411
6581 25896 contig-100_5 56 15154 15207
26147 50433 contig-100_3 32 24270 24230
50399 60617 contig-100_4 0 9511 9497
53285 67570 contig-100_4 7 10394 17154
67552 69636 contig-100_20 0 2084 2067
70162 98420 contig-100_2 0 28209 28206
98405 103668 contig-100_11 0 5248 5248
103570 148362 contig-100_0 0 44340 44344
148367 163827 contig-100_6 48 15113 15162
163793 196751 contig-100_1 54 32920 32883
196722 208040 contig-100_7 3 11490 11428


In [113]:
def find_overlap(b1, b2, spacer=50):
    delta = b1.msa_stop - b2.msa_start 
    b1_seq = b1.seq[-(delta + spacer):]
    b2_seq = b2.seq[:delta + spacer]
    aln = perform_aln(b1_seq, b2_seq)
    if len(aln) == 0:
        s = b1.seq[:-delta]
        assert len(s) > 0
        return s
    else:
        psl = sorted([PslRow(x.split('\t')) for x in aln], key=lambda x: x.coverage)[-1]
        cutoff = psl.query_coordinate_to_target(psl.q_end - 1) + 1
        return b1.seq[:-cutoff]

def merge_same(b1, b2):
    with TemporaryFilePath() as f, TemporaryFilePath() as f2:
        with open(f, 'w') as outf:
            write_fasta(outf, 'b1', b1.seq)
            write_fasta(outf, 'b2', b2.seq)
        cmd = ['muscle', '-in', f, '-out', f2]
        run_proc(cmd)
        fa = Fasta(f2)
        seq = []
        for x, y in zip(*[fa['b1'], fa['b2']]):
            if x == '-':
                seq.append(y)
            else:
                seq.append(x)
        return ''.join(seq)
    
def perform_aln(b1_seq, b2_seq):
    with TemporaryFilePath() as b1_f, TemporaryFilePath() as b2_f:
        with open(b1_f, 'w') as b1_f_h:
            write_fasta(b1_f_h, 'b1', b1_seq)
        with open(b2_f, 'w') as b2_f_h:
            write_fasta(b2_f_h, 'b1', b2_seq)
        cmd = ['blat', b1_f, b2_f, '-noHead', '/dev/stdout']
        return call_proc_lines(cmd)[:-2]

In [186]:
seqs = {}
for name, mb in filtered_blocks.iteritems():
    seq = []
    a, b = tee(mb)
    _ = next(b, None)
    merged = set()
    for i, (b1, b2) in enumerate(izip(a, b)):
        if b1.qname in merged:
                continue
        elif b1.qname == b2.qname:
            seq.append(merge_same(b1, b2))
            merged.add(b1.qname)
        elif b2.msa_start < b1.msa_stop:  # we have an overlap, resolve via pairwise alignment
            seq.append(find_overlap(b1, b2))
        elif b1.msa_stop == b2.msa_start:  # exactly contiguous, just add b1 and continue
            seq.append(b1.seq)
        else:  # we have a unknown gap, so add b1 then a gap
            seq.append(b1.seq)
            seq.append(''.join(['N'] * 100))
        # add the final b2 sequence
    seq.append(b2.seq)  # add the last sequence
    seqs[name] = seq

In [22]:
with open('a2.contigs.fa', 'w') as outf:
    for i, x in enumerate(seqs['A2']):
        write_fasta(outf, str(i), x)

In [339]:
%mkdir -p assemblies_v2
from tools.procOps import *
for name, seq in seqs.iteritems():
    with open('assemblies_v2/{}.fa'.format(name), 'w') as outf:
        write_fasta(outf, name, ''.join(seq))
    cmd = [['bwa', 'mem', '-t', '4', '-x', 'intractg', '/hive/groups/recon/10x_genomics/references/refdata-hsapiens-hg38/fasta/genome.fa', 
            'assemblies_v2/{}.fa'.format(name)],
           ['samtools', 'view', '-b', '-'],
           ['sambamba', 'sort', '-o', 'assemblies_v2/{}.hg38.bam'.format(name), '/dev/stdin']]
    run_proc(cmd)

In [115]:
name = 'A2'
mb = filtered_blocks[name]
seq = []
a, b = tee(mb)
_ = next(b, None)
merged = set()
for i, (b1, b2) in enumerate(izip(a, b)):
    if b1.qname in merged:
        continue
    print i, b1.qname, b2.qname
    if b1.qname == b2.qname:
        print 'merge', len(m)
        m = merge_same(b1, b2)
        if name in ['A1', 'A2', 'B1', 'B2', 'N1', 'N2']:
            m = reverse_complement(m)
        seq.append(m)
        merged.add(b1.qname)
    elif b2.msa_start < b1.msa_stop:  # we have an overlap, try to resolve via pairwise alignment
        o = find_overlap(b1, b2)
        print 'overlap', len(o)
        if name in ['A1', 'A2', 'B1', 'B2', 'N1', 'N2']:
            o = reverse_complement(o)
        seq.append(o)
    elif b1.msa_stop == b2.msa_start:
        print 'same start/stop', len(b1.seq)
        if name in ['A1', 'A2', 'B1', 'B2', 'N1', 'N2']:
            s = reverse_complement(b1.seq)
        else:
            s = b1.seq
        seq.append(s)
    else:
        if name in ['A1', 'A2', 'B1', 'B2', 'N1', 'N2']:
            s = reverse_complement(b1.seq)
        else:
            s = b1.seq
        seq.append(s)
        seq.append(''.join(['N'] * 100))
        print 'gap', len(b1.seq)
if name in ['A1', 'A2', 'B1', 'B2', 'N1', 'N2']:
    s = reverse_complement(b2.seq)
else:
    s = b2.seq
seq.append(s)  # add the last sequence

0 contig-100_16 contig-100_5
overlap 2387
1 contig-100_5 contig-100_3
gap 15207
2 contig-100_3 contig-100_4
overlap 24196
3 contig-100_4 contig-100_4
merge 6228
5 contig-100_20 contig-100_2
gap 2067
6 contig-100_2 contig-100_11
overlap 28191
7 contig-100_11 contig-100_0
overlap 5100
8 contig-100_0 contig-100_6
gap 44344
9 contig-100_6 contig-100_1
overlap 15128
10 contig-100_1 contig-100_7
overlap 32854


In [129]:
b = blocks['A2']
mb = []
for start, end, qname, seq, seq_start, seq_end in b:
    closest_start = find_closest(sorted_positions, start)
    closest_stop = find_closest(sorted_positions, end)
    msa_start = final_map[closest_start]
    msa_stop = final_map[closest_stop]
    if msa_start > msa_stop:  # handle negative strand
        msa_start, msa_stop = msa_stop, msa_start
        seq = reverse_complement(seq)
    assert len(seq) > 0
    mb.append(Record(msa_start, msa_stop, qname, seq, seq_start, seq_end))
# sort these to be in notch2nl order
mb = sorted(mb, key=lambda x: x.msa_start)

In [202]:
# extract all alignments, discarding those not in our ROIs
b = []
for aln in pysam.Samfile(bam):
    if aln.is_unmapped:
        continue
    start = aln.reference_start
    end = aln.reference_end
    c = ChromosomeInterval('chr1', start, end, '.')
    if not interval_not_intersect_intervals(regions_of_interest, c) and len(c) > 100:
        b.append([start, end, aln.qname, aln.seq, aln.qstart, aln.qstart + aln.alen])

In [16]:
aln_f = Fasta(n2nl_aln)
seq_aln_map = defaultdict(dict)
for name, seq in aln_f.iteritems():
    seq_pos = 0
    for aln_pos, x in enumerate(str(seq)):
        seq_aln_map[name][seq_pos] = aln_pos
        if x != '-':
            seq_pos += 1

# find maximum position for reversing negative strand
max_pos = {x: max(y.keys()) for x, y in seq_aln_map.iteritems()}

# construct a hg38 -> sequence positions using the sequences trivially mapped back to hg38
hg38_map = {}
for rec in pysam.Samfile(hg38_bam):
    m = {y: x for x, y in rec.aligned_pairs}
    # invert positions for negative strand genes
    if rec.qname in ['NOTCH2', 'NOTCH2NL-A', 'NOTCH2NL-B']:
        m = {x: max_pos[rec.qname] - y for x, y in m.iteritems()}
    hg38_map[rec.qname] = m

# construct a table mapping each alignment position to all hg38 positions
r = defaultdict(dict)
for name, pos_map in hg38_map.iteritems():
    for hg38_pos, seq_pos in pos_map.iteritems():
        aln_pos = seq_aln_map[name][seq_pos]
        r[name][aln_pos] = hg38_pos

# now invert this map, so that we have our hg38 -> aln map
final_map = {}
for name in r:
    for aln_pos in r[name]:
        hg38_pos = r[name][aln_pos]
        assert hg38_pos not in final_map
        final_map[hg38_pos] = aln_pos


KeyError: 11